In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly_express as px
import json
import plotly.offline as offline
from skimage import io
import dash
from dash import html
from dash.dependencies import Input, Output
import numpy as np
import mysql.connector
import itertools

cnx = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='ddb')

cursor = cnx.cursor(buffered=True)
result_before={}
result_after={}
temp=[]
list_DIS_factor_before=['PDI05HIJ','PDIS_05A','PDIS_05B','PDIS_05C','PDIS_05D','PDIS_05L','DIS_05_E','DIS_05_F','DIS_05_G']
list_DIS_factor_after=['PDI10HIJ','PDIS_10A','PDIS_10B','PDIS_10C','PDIS_10D','PDIS_10L','DIS_10_E','DIS_10_F','DIS_10_G']
list_SIT_factor_before=['DTS_05_A','DTS_05_B','DTS_05_C','DTS_05_D','DTS_05_E','DTS_05_F','DTS_05_G','DTS_05_H','DTS_05_I','DTS_05_J','DTS_05_K']
list_SIT_factor_after=['DTS_10_A','DTS_10_B','DTS_10_C','DTS_10_D','DTS_10_E','DTS_10_F','DTS_10_G','DTS_10_H','DTS_10_I','DTS_10_J','DTS_10_K']
for dis_before,sit_before in itertools.product(list_DIS_factor_before, list_SIT_factor_before):
    query = ("SELECT `dis-rate` FROM( SELECT "+sit_before+", count(*) * 100.0 / sum(count(*)) over() as `dis-rate`"
    "from icced2020rcced_p WHERE "+dis_before+"=1   group by "+sit_before+") AS B WHERE "+sit_before+"=1;")
    cursor.execute(query)
    rowCount = cursor.fetchone()
    try:
        result_before[dis_before].append(float(rowCount[0]))
    except KeyError:
        result_before[dis_before]=[]
        result_before[dis_before].append(float(rowCount[0]))
for dis_after,sit_after in itertools.product(list_DIS_factor_after, list_SIT_factor_after):
    query = ("SELECT `dis-rate` FROM( SELECT "+sit_after+", count(*) * 100.0 / sum(count(*)) over() as `dis-rate`"
    "from icced2020rcced_p WHERE "+dis_after+"=1   group by "+sit_after+") AS B WHERE "+sit_after+"=1;")
    cursor.execute(query)
    rowCount = cursor.fetchone()
    try:
        result_after[dis_after].append(float(rowCount[0]))
    except KeyError:
        result_after[dis_after]=[]
        result_after[dis_after].append(float(rowCount[0]))
cursor.close()
cnx.close()

# sense of belonging data
cnx = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='ddb')
cursor = cnx.cursor(buffered=True)
result={}
temp=[]
list_DIS_factor_before=['PDI05HIJ','PDIS_05A','PDIS_05B','PDIS_05C','PDIS_05D','PDIS_05L','DIS_05_E','DIS_05_F','DIS_05_G']
list_DIS_factor_after=['PDI10HIJ','PDIS_10A','PDIS_10B','PDIS_10C','PDIS_10D','PDIS_10L','DIS_10_E','DIS_10_F','DIS_10_G']
list_community=['BEL_05J','BEL_05E','BEL_05D','BEL_05A']
list_bel_strength=[1,2,3,4]
for com,dis,strength in itertools.product(list_community,zip(list_DIS_factor_before, list_DIS_factor_after),list_bel_strength):
    query = ("SELECT `SB` FROM( SELECT "+com+", count(*) * 100.0 / sum(count(*)) over() as `SB`"
"from icced2020rcced_p WHERE "+dis[0]+"=1 OR "+dis[1]+"=1   group by "+com+") AS B WHERE "+com+"=%s;")
    cursor.execute(query,(strength,))
    rowCount = cursor.fetchone()
    if rowCount is None:
        rowCount=(0,)
    try:
        result[dis[0]+"_"+com].append(str(rowCount[0]))
    except KeyError:
        result[dis[0]+"_"+com]=[]
        result[dis[0]+"_"+com].append(str(rowCount[0]))
cursor.close()
cnx.close()

# map data
cnx = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='ddb')
cursor = cnx.cursor(buffered=True)
result_difference={}
temp=[]
list_DIS_factor_before=['PDI05HIJ','PDIS_05A','PDIS_05B','PDIS_05C','PDIS_05D','PDIS_05L','DIS_05_E','DIS_05_F','DIS_05_G']
list_DIS_factor_after=['PDI10HIJ','PDIS_10A','PDIS_10B','PDIS_10C','PDIS_10D','PDIS_10L','DIS_10_E','DIS_10_F','DIS_10_G']
list_PROV=[48,59,46,13,10,61,12,35,11,24,47,60,62]
for dis,PROV in itertools.product(zip(list_DIS_factor_before, list_DIS_factor_after), list_PROV):
    query = ("SELECT `dis-rate-after`-`dis-rate-before` AS DIFFERENCE FROM"
    "(SELECT "+dis[1]+", count(*) * 100.0 / sum(count(*)) over() as `dis-rate-after`"
    "from icced2020rcced_p WHERE PPROV=%s  group by "+dis[1]+") AS `after-COV` INNER JOIN"
    "(SELECT "+dis[0]+", count(*) * 100.0 / sum(count(*)) over() as `dis-rate-before`"
    "from icced2020rcced_p WHERE PPROV=%s  group by "+dis[0]+") AS `before-COV` ON `before-COV`."+dis[0]+"=`after-COV`."+dis[1]+" WHERE "+dis[0]+"=1 ;")
    cursor.execute(query,(PROV,PROV))
    rowCount = cursor.fetchone()
    if rowCount is None:
        rowCount=(0,)
    try:
        result_difference[dis[0]].append(str(rowCount[0]))
    except KeyError:
        result_difference[dis[0]]=[]
        result_difference[dis[0]].append(str(rowCount[0]))
cursor.close()
cnx.close()

pd.options.plotting.backend = "plotly"

# polar data
categories = ['Store/bank/rest', 'School', 'On the internet', 'Work', 'Housing', 'Police', 'Courts', 'Border onto Canada', 
             'Social gatherings', 'Public areas', 'Public transit', 'Other']

# make subplots
fig = make_subplots(
    #shared_xaxes=True,
    vertical_spacing=0.1,
    #column_widths=[1],
    #row_heights=[1, 0.6, 1],
    specs=[[{"type": "scatterpolar"}],
           [{"type": "parcats"}],
           [{"type": "image"}]],
    rows=3, cols=1)

# polar title
fig.update_layout(
    title={
        'text': "<b>Impacts of COVID-19 on Canadians' Experiences of Discrimination</b>",
        'y':0.998,
        'x':0.29,
        'xanchor': 'left',
        'yanchor': 'top'})

#Sense of belonging
cnx = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='ddb')
result={}
temp=[]
list_DIS_factor_before=['PDI05HIJ','PDIS_05A','PDIS_05B','PDIS_05C','PDIS_05D','PDIS_05L','DIS_05_E','DIS_05_F','DIS_05_G']
list_DIS_factor_after=['PDI10HIJ','PDIS_10A','PDIS_10B','PDIS_10C','PDIS_10D','PDIS_10L','DIS_10_E','DIS_10_F','DIS_10_G']
list_community=['BEL_05J','BEL_05E','BEL_05D','BEL_05A']
list_bel_strength=[1,2,3,4]
count = 0
for dis in zip(list_DIS_factor_before,list_DIS_factor_after):
    query = ("SELECT * FROM (SELECT * from icced2020rcced_p WHERE BEL_05J!=5 AND BEL_05J!=9  AND BEL_05E!=5 AND BEL_05E!=9 AND BEL_05D!=5 AND BEL_05D!=9 AND BEL_05A!=5 AND BEL_05A!=9) AS A WHERE "+dis[0]+"=1 OR "+dis[1]+"=1;")
    data_df = pd.read_sql(query, cnx)
    BEL_05A_dim = go.parcats.Dimension(
    values=data_df.BEL_05A, label="Local community", categoryarray=[1, 2, 3, 4],
    ticktext=['Very weak', 'Somewhat weak', 'Somewhat strong', 'Very strong']
    )
    BEL_05D_dim = go.parcats.Dimension(
    values=data_df.BEL_05D, label="Canada", categoryarray=[1, 2, 3, 4],
    ticktext=['Very weak', 'Somewhat weak', 'Somewhat strong', 'Very strong']
    )
    BEL_05E_dim = go.parcats.Dimension(
    values=data_df.BEL_05E, label="Country of origin", categoryarray=[1, 2, 3, 4],
    ticktext=['Very weak', 'Somewhat weak', 'Somewhat strong', 'Very strong']
    )
    BEL_05J_dim = go.parcats.Dimension(
    values=data_df.BEL_05J, label="Online communities most active in", categoryarray=[1, 2, 3, 4],
    ticktext=['Very weak', 'Somewhat weak', 'Somewhat strong', 'Very strong']
    )
    # Create parcats trace
    color = data_df.BEL_05A;
    colorscale = [[0, 'red'], [1, 'blue']];
    if count == 0:
        fig.add_trace(go.Parcats(dimensions=[BEL_05A_dim, BEL_05D_dim, BEL_05E_dim, BEL_05J_dim],
            line={'color': color, 'colorscale': colorscale},
            hoveron='color', hoverinfo='count+probability',
            labelfont={'size': 18, 'family': 'Times'},
            tickfont={'size': 16, 'family': 'Times'},
            arrangement='freeform',
            visible=True), 2, 1)
    else:
        fig.add_trace(go.Parcats(dimensions=[BEL_05A_dim, BEL_05D_dim, BEL_05E_dim, BEL_05J_dim],
            line={'color': color, 'colorscale': colorscale},
            hoveron='color', hoverinfo='count+probability',
            labelfont={'size': 18, 'family': 'Times'},
            tickfont={'size': 16, 'family': 'Times'},
            arrangement='freeform',
            visible=False), 2, 1)
    count = count + 1
cursor.close()
cnx.close()

# Discrimination – Sex, sexual orientation, gender identity/expression
# Add polar trace
fig.add_trace(go.Scatterpolar(
      r=result_before['PDI05HIJ'],
      theta=categories,
      fill='none',
      line_color = 'blue',
      name='2 yrs before COVID-19',
      visible=True), 1, 1)

fig.add_trace(go.Scatterpolar(
      r=result_after['PDI10HIJ'],
      theta=categories,
      fill='none',
      line_color = 'red',
      name='Since COVID-19 began', 
      visible=True), 1, 1)

# Discrimination - Indigenous identity
# Add polar trace
fig.add_trace(go.Scatterpolar(
      r=result_before['PDIS_05A'],
      theta=categories,
      fill='none',
      line_color = 'blue',
      name='2 yrs before COVID-19',
      visible=False), 1, 1)

fig.add_trace(go.Scatterpolar(
      r=result_after['PDIS_10A'],
      theta=categories,
      fill='none',
      line_color = 'red',
      name='Since COVID-19 began', 
      visible=False), 1, 1)

# Discrimination - Ethnicity/culture
# Add polar trace
fig.add_trace(go.Scatterpolar(
      r=result_before['PDIS_05B'],
      theta=categories,
      fill='none',
      line_color = 'blue',
      name='2 yrs before COVID-19',
      visible=False), 1, 1)

fig.add_trace(go.Scatterpolar(
      r=result_after['PDIS_10B'],
      theta=categories,
      fill='none',
      line_color = 'red',
      name='Since COVID-19 began', 
      visible=False), 1, 1)

# Discrimination - Race/skin colour
# Add polar trace
fig.add_trace(go.Scatterpolar(
      r=result_before['PDIS_05C'],
      theta=categories,
      fill='none',
      line_color = 'blue',
      name='2 yrs before COVID-19',
      visible=False), 1, 1)

fig.add_trace(go.Scatterpolar(
      r=result_after['PDIS_10C'],
      theta=categories,
      fill='none',
      line_color = 'red',
      name='Since COVID-19 began', 
      visible=False), 1, 1)

# Discrimination - Religion
# Add polar trace
fig.add_trace(go.Scatterpolar(
      r=result_before['PDIS_05D'],
      theta=categories,
      fill='none',
      line_color = 'blue',
      name='2 yrs before COVID-19',
      visible=False), 1, 1)

fig.add_trace(go.Scatterpolar(
      r=result_after['PDIS_10D'],
      theta=categories,
      fill='none',
      line_color = 'red',
      name='Since COVID-19 began', 
      visible=False), 1, 1)

# Discrimination - Physical/mental disability
# Add polar trace
fig.add_trace(go.Scatterpolar(
      r=result_before['PDIS_05L'],
      theta=categories,
      fill='none',
      line_color = 'blue',
      name='2 yrs before COVID-19',
      visible=False), 1, 1)

fig.add_trace(go.Scatterpolar(
      r=result_after['PDIS_10L'],
      theta=categories,
      fill='none',
      line_color = 'red',
      name='Since COVID-19 began', 
      visible=False), 1, 1)

# Discrimination - Language
# Add polar trace
fig.add_trace(go.Scatterpolar(
      r=result_before['DIS_05_E'],
      theta=categories,
      fill='none',
      line_color = 'blue',
      name='2 yrs before COVID-19',
      visible=False), 1, 1)

fig.add_trace(go.Scatterpolar(
      r=result_after['DIS_10_E'],
      theta=categories,
      fill='none',
      line_color = 'red',
      name='Since COVID-19 began',  
      visible=False), 1, 1)

# Discrimination - Accent
# Add polar trace
fig.add_trace(go.Scatterpolar(
      r=result_before['DIS_05_F'],
      theta=categories,
      fill='none',
      line_color = 'blue',
      name='2 yrs before COVID-19',
      visible=False), 1, 1)

fig.add_trace(go.Scatterpolar(
      r=result_after['DIS_10_F'],
      theta=categories,
      fill='none',
      line_color = 'red',
      name='Since COVID-19 began', 
      visible=False), 1, 1)

# Discrimination - Physical appearance
# Add polar trace
fig.add_trace(go.Scatterpolar(
      r=result_before['DIS_05_G'],
      theta=categories,
      fill='none',
      line_color = 'blue',
      name='2 yrs before COVID-19',
      visible=False), 1, 1)

fig.add_trace(go.Scatterpolar(      
      r=result_after['DIS_10_G'],
      theta=categories,
      fill='none',
      line_color = 'red',
      name='Since COVID-19 began', 
      visible=False), 1, 1)

img = io.imread('https://raw.githubusercontent.com/alirezat137886/discriminationdata/main/plot%201.png')

fig.add_trace(go.Image(z=img, visible=True), 3, 1)

img = io.imread('https://raw.githubusercontent.com/alirezat137886/discriminationdata/main/plot%202.png')

fig.add_trace(go.Image(z=img, visible=False), 3, 1)

img = io.imread('https://raw.githubusercontent.com/alirezat137886/discriminationdata/main/plot%203.png')

fig.add_trace(go.Image(z=img, visible=False), 3, 1)

img = io.imread('https://raw.githubusercontent.com/alirezat137886/discriminationdata/main/plot%204.png')

fig.add_trace(go.Image(z=img, visible=False), 3, 1)

img = io.imread('https://raw.githubusercontent.com/alirezat137886/discriminationdata/main/plot%205.png')

fig.add_trace(go.Image(z=img, visible=False), 3, 1)

img = io.imread('https://raw.githubusercontent.com/alirezat137886/discriminationdata/main/plot%206.png')

fig.add_trace(go.Image(z=img, visible=False), 3, 1)

img = io.imread('https://raw.githubusercontent.com/alirezat137886/discriminationdata/main/plot%207.png')

fig.add_trace(go.Image(z=img, visible=False), 3, 1)

img = io.imread('https://raw.githubusercontent.com/alirezat137886/discriminationdata/main/plot%208.png')

fig.add_trace(go.Image(z=img, visible=False), 3, 1)

img = io.imread('https://raw.githubusercontent.com/alirezat137886/discriminationdata/main/plot%209.png')

fig.add_trace(go.Image(z=img, visible=False), 3, 1)

button1 = dict(method='update', 
               args=[{"visible": [True, False, False, False, False, False, False, False, False,
                                  True, True, False, False, False, False, False, False, False,
                                  False, False, False, False, False, False, False, False, False,
                                  True, False, False, False, False, False, False, False, False]}], 
               label="Sex, sexual orientation, gender identity/expression")
button2 = dict(method='update', 
               args=[{"visible": [False, True, False, False, False, False, False, False, False,
                                  False, False, True, True, False, False, False, False, False,
                                  False, False, False, False, False, False, False, False, False,
                                  False, True, False, False, False, False, False, False, False]}],
               label="Indigenous identity")   
button3 = dict(method='update', 
               args=[{"visible": [False, False, True, False, False, False, False, False, False,
                                  False, False, False, False, True, True, False, False, False,
                                  False, False, False, False, False, False, False, False, False, 
                                  False, False, True, False, False, False, False, False, False]}],
               label="Ethnicity/culture")  
button4 = dict(method='update', 
               args=[{"visible": [False, False, False, True, False, False, False, False, False,
                                  False, False, False, False, False, False, True, True, False,
                                  False, False, False, False, False, False, False, False, False, 
                                  False, False, False, True, False, False, False, False, False]}],
               label="Race/skin colour")  
button5 = dict(method='update', 
               args=[{"visible": [False, False, False, False, True, False, False, False, False,
                                  False, False, False, False, False, False, False, False, True,
                                  True, False, False, False, False, False, False, False, False,  
                                  False, False, False, False, True, False, False, False, False]}],
               label="Religion")  
button6 = dict(method='update', 
               args=[{"visible": [False, False, False, False, False, True, False, False, False,
                                  False, False, False, False, False, False, False, False, False,
                                  False, True, True, False, False, False, False, False, False, 
                                  False, False, False, False, False, True, False, False, False]}],
               label="Physical/mental disability")  
button7 = dict(method='update', 
               args=[{"visible": [False, False, False, False, False, False, True, False, False,
                                  False, False, False, False, False, False, False, False, False,
                                  False, False, False, True, True, False, False, False, False,  
                                  False, False, False, False, False, False, True, False, False]}],
               label="Language")  
button8 = dict(method='update', 
               args=[{"visible": [False, False, False, False, False, False, False, True, False,
                                  False, False, False, False, False, False, False, False, False,
                                  False, False, False, False, False, True, True, False, False, 
                                  False, False, False, False, False, False, False, True, False]}],
               label="Accent")  
button9 = dict(method='update', 
               args=[{"visible": [False, False, False, False, False, False, False, False, True,
                                  False, False, False, False, False, False, False, False, False,
                                  False, False, False, False, False, False, False, True, True, 
                                  False, False, False, False, False, False, False, False, True]}],
               label="Physical appearance")

fig.update_layout(width=1300, height=2500, autosize=False, margin=dict(t=200, b=0, l=100, r=0))
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

fig.update_layout(updatemenus =[dict(type='dropdown',
                                    buttons=[button1, button2, button3, button4, button5, button6, button7, button8, button9],
                                    direction="down",
                                    #pad={"b": 100},
                                    showactive=True,
                                    x=-0.2,
                                    xanchor="left",
                                    y=1.07,
                                    yanchor="top"
                                    )])

fig.add_annotation(dict(font=dict(color='black',size=16),
                                        x=0.30,
                                        y=0.67,
                                        showarrow=False,
                                        text="<b>Sense of Belonging to Different Communities</b>",
                                        textangle=0,
                                        xanchor='left',
                                        yanchor='top',
                                        xref="paper",
                                        yref="paper"))

fig.add_annotation(dict(font=dict(color='black', size=16),
                                        x=0.30,
                                        y=1.035,
                                        showarrow=False,
                                        text="<b>Discrimination % rate in Different Situations</b>",
                                        textangle=0,
                                        xanchor='left',
                                        yanchor='top',
                                        xref="paper",
                                        yref="paper"))

fig.add_annotation(dict(font=dict(color='black', size=14),
                                        x=-0.20,
                                        y=1.087,
                                        showarrow=False,
                                        text="Discrimination factor:",
                                        textangle=0,
                                        xanchor='left',
                                        yanchor='top',
                                        xref="paper",
                                        yref="paper"))

offline.plot(fig, filename='tmp.html')

ModuleNotFoundError: No module named 'plotly_express'